## Lab 13.b -- Crime Stats

As before, our code below loads the data, and purges the "clearly wrong" records. Your lab tasks appear towards the end of the notebook.


In [ ]:
import json
import os
  
## please load the data from the mp248-course-notes repository.
## It is a large file.  i.e. : Please **DO NOT** store in your personal repo, i.e.
## please do not copy this file, nor move this file.

## the code below should execute in your notebook -- if not, modify the directory
## accordingly.

with open("../data/Victoria (BC) Police Department.geojson") as f:
    data = json.loads(f.read())
    

In [ ]:
import collections as co
import pprint as pp

ctree = dict()

for R in data['features']:
    TopI = R['properties']['parent_incident_type']
    BotI = R['properties']['incident_type_primary']
    if TopI not in ctree.keys():
        ctree[TopI] = co.defaultdict(int)
    ctree[TopI][BotI] += 1
    
pp.pprint(ctree)


In [ ]:
import datetime as dt

clean_dat = list()
incompl_dat = list()

for x in data['features']:
    if 'properties' not in x.keys():
        continue ## why bother?

    ## we have the properties, so let's start building the record.
    newRec = dict()
    incFlag = False # set to true if we discover missing data in x
    
    floatkeys = ['latitude', 'longitude']
    strkeys = ['parent_incident_type', 'incident_type_primary', 'address_1', 'address_2']
    intkeys = ['case_number', 'incident_id']
    datekeys = ['created_at', 'updated_at', 'incident_datetime']
    
    for k in floatkeys:
        if k in x['properties'].keys() and isinstance(x['properties'][k], str) and\
        len(x['properties'][k])>0:
            newRec[k] = float(x['properties'][k])
        else:
            incFlag=True

    ## anything to check?
    for k in strkeys:
        newRec[k] = x['properties'][k]
        
    for k in intkeys:
        ## let's count non-numerical characters
        non_ch_chars = []
        digits = '0123456789'
        for ch in x['properties'][k]:
            if ch not in digits:
                non_ch_chars.append(ch)

        for ch in non_ch_chars:
            if ch != '-':
                x['properties'][k] = x['properties'][k].replace(ch, '-')
            
        if '-' in x['properties'][k]:
            newRec[k] = [int(w) for w in x['properties'][k].split('-')]
        else:
            newRec[k] = int(x['properties'][k])
            
    for k in datekeys:
        ## typical format: '2014-12-06T21:59:00.000'
        newRec[k] = dt.datetime.strptime(x['properties'][k], '%Y-%m-%dT%H:%M:%S.000')
        
    if incFlag:
        incompl_dat.append(newRec)
    else:
        clean_dat.append(newRec)

In [ ]:
## Let's use the results from the lab earlier this week to fix or purge the clearly bad data from
## clean_dat...

bad_idx = []

for i in range(len(clean_dat)):
    x = clean_dat[i]
    if x['latitude']>48.655 or x['latitude']<48:
        bad_idx.append(i)
        continue
    if x['longitude']<-124 or x['longitude']>-123:
        bad_idx.append(i)
        continue
    if x['incident_datetime'] > x['created_at']:
        bad_idx.append(i)
        continue
    if x['updated_at'] < x['created_at']:
        bad_idx.append(i)
print("Original number of records in clean_dat: ", len(clean_dat))
print("Bad records discovered: ", len(bad_idx))

clean_dat = [clean_dat[i] for i in range(len(clean_dat)) if i not in bad_idx]
print("Purged clean_dat: ", len(clean_dat))

**Problem 1:** Write a Python function that uses spherical polar coordinates to determine the distance between any two points on the earth's surface.  The function will take as input two points, given in spherical polar coordinates, i.e. *longitude* and *latitude*. Assume your points are on the earth's surface, and that the earth is a sphere of radius **6378**km. 

As a sanity check, find the GPS coordinates of two points on the earth for which you know the distance "as the crow flies".  Compute their distance using your function, and compare.

**Problem 2:** Find the longitude and latitude of *The Bay Centre* in downtown Victoria. *Hint:* If you zoom in on something in Google maps, the longitude and latitude should appear in the address bar.  If this does not happen in your browser, try the *share* option. 

**Problem 3:** If we use the GPS coordinates of the Bay Centre as a proxy for the *centre of downtown Victoria*, what relation is there between the quantity of Victoria PD crime records and their distance to the centre of downtown?   One way to answer this problem would be to build a histogram where the x-axis is the distance to downtown, and the y-axis is the crime count.  Perhaps choose your bin size so that all your bins have equal area. 


**Problem 4:** Compute the latitude and longitude of the average position of all the crimes in **clean_dat**. Look up that location in Google Maps.  How would you describe it, is it a store, or perhaps its in the middle of a street?
    